In [ ]:
!pip install langchain==0.2.11 langchain-community==0.2.10 langchain-text-splitters==0.2.2 langchain-groq==0.1.6 transformers==4.43.2 sentence-transformers==3.0.1 unstructured==0.15.0 unstructured[pdf]==0.15.0 PyPDF2==3.0.1 python-dotenv==1.0.0 streamlit==1.18.1 openai==0.27.6 faiss-cpu==1.7.4 altair==4 tiktoken==0.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 20.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
IN

In [2]:
import streamlit as st
from dotenv import load_dotenv
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings,HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import HuggingFaceHub
from langchain_community.document_loaders import WebBaseLoader
import bs4
import os
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA

ModuleNotFoundError: No module named 'streamlit'

In [ ]:
%%writefile app.py
#Create VectorDB Function
import streamlit as st
from dotenv import load_dotenv
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings,HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import HuggingFaceHub
from langchain_community.document_loaders import WebBaseLoader
import bs4
import os
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
# '''
# This function create an vectorDB which save all the data you want to ask about in a database using vectors this speedup search and help LLM understand the data
# you can adjest the type of embedding for olamaa embedding or openai or hugging face embedding ,etc
# and save vectordb in a folder called VectorDB
# pdf path and embedding as parameters
# return none
# '''
embeddings = HuggingFaceEmbeddings()


def Create_vectordb(PDF,embeddings):
  # loading the document
  text = ""
  pdf_reader = PdfReader(PDF)
  for page in pdf_reader.pages:
    text += page.extract_text()

  # splitting into text chunks
  # text_splitter = CharacterTextSplitter(
  #     separator="\n",
  #     chunk_size=2000,
  #     chunk_overlap=400,
  #     length_function=len
  # )
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=300)

  texts = text_splitter.split_text(text)
  vectordb = FAISS.from_texts(texts=texts
                              ,embedding=embeddings)
  return vectordb


Overwriting app.py


In [ ]:
%%writefile app.py -a
#load VectorDB Function
# '''
# Load VectorDB

# parameters
# Folder path Location of the vectorDB in disk
# Embeddings that embedding that was used while creating the vectordb

# return
# VectorDB variable
# '''
def Load_vectordb(Folder_Path,embeddings):
  return FAISS.load_local(folder_path=Folder_Path,embeddings=embeddings,allow_dangerous_deserialization=True)




Appending to app.py


In [ ]:
%%writefile app.py -a
#save vectorDB function
# '''
# save VectorDB

# parameters
# VectorDB takes the vectorDB you want to save as a variable
# Folder path Location of the vectorDB in disk


# return
# none
# '''
def save_vectordb(vectordb,Folder_Path):
   vectordb.save_local(folder_path=Folder_Path)
   print("save successful")



Appending to app.py


In [ ]:
%%writefile app.py -a
#update VectorDB function
# '''
# Updates the vectorDB with new data
# size of the chunk is 2000 Characters and 400 overlaps

# available data Types (PDF,Website from wikiPedia)

# parameters
# Folder_path: Location of the vectorDB in disk
# Embeddings: that embedding that was used while creating the vectordb
# Data: path PDF or the link of website
# Type:1--> PDF   2--> website link
# '''
def Update_VectorDB(Folder_Path,embeddings,Data,Type):
  vectordb = Load_vectordb(Folder_Path,embeddings)

  # splitting into text chunks
  if Type== 1:
    text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1500,
    chunk_overlap=300,
    length_function=len)
    text = ""
    pdf_reader = PdfReader(Data)
    for page in pdf_reader.pages:
      text += page.extract_text()
    texts = text_splitter.split_text(text)
    vectordb.add_texts(texts)
    print("VectorDB updated")
  if Type == 2:
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=300)
    loader = WebBaseLoader(web_paths=(Data,),
                          bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                              class_=("mw-body"))))
    text_documents = loader.load()

    # Convert the list to an iterable
    documents = text_splitter.split_documents(text_documents)
    vectordb.add_documents(documents)
  return vectordb




Appending to app.py


In [ ]:
%%writefile app.py -a
#function to load_LLM using Groq
# '''
# deploy the LLM model using groq api

# parameters
# vectordb take the vectordb where all the data in stored
# model_name the type of LLM you want to use in groq api here is link of all available models
# '''
#add groq key
def load_LLM_groq(vectordb,Model_name):
  retriever = vectordb.as_retriever()
  llm = ChatGroq(
    model=Model_name,
    temperature=0)
  qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True)
  return qa_chain



Appending to app.py


In [ ]:
def load_LLM_groq(vectordb,Model_name):
  retriever = vectordb.as_retriever()
  llm = ChatGroq(
    model=Model_name,
    temperature=0)
  qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True)
  return qa_chain

In [ ]:
%%writefile app.py -a
#function to Load_LLM using OpenAI
# '''
# deploy the LLM model using OpenAI api

# parameters
# vectordb take the vectordb where all the data in stored
# model_name the type of LLM you want to use in OpenAI api here is link of all available models
# '''
# Add OpenAI API key
# os.environ["OPENAI_API_KEY"] = "your_openai_api_key"
def load_LLM_openai(vectordb, model_name="gpt-4"):
    retriever = vectordb.as_retriever()

    # Initialize OpenAI model
    llm = OpenAI(
        model_name=model_name,  # Example: "gpt-4o" or "gpt-3.5-turbo"
        temperature=0
    )

    # Set up RetrievalQA chain with OpenAI model
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",  # Use 'stuff', 'map_reduce', etc. as needed
        retriever=retriever,
        return_source_documents=True
    )

    return qa_chain


Appending to app.py


In [ ]:
embeddings = HuggingFaceEmbeddings()


def Create_vectordb(PDF,embeddings):
  # loading the document
  text = ""
  pdf_reader = PdfReader(PDF)
  for page in pdf_reader.pages:
    text += page.extract_text()

  # splitting into text chunks
  # text_splitter = CharacterTextSplitter(
  #     separator="\n",
  #     chunk_size=2000,
  #     chunk_overlap=400,
  #     length_function=len
  # )
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=300)

  texts = text_splitter.split_text(text)
  vectordb = FAISS.from_texts(texts=texts
                              ,embedding=embeddings)
  return vectordb


def Update_VectorDB(Folder_Path,embeddings,Data,Type):
  vectordb = Load_vectordb(Folder_Path,embeddings)

  # splitting into text chunks
  if Type== 1:
    text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1500,
    chunk_overlap=300,
    length_function=len)
    text = ""
    pdf_reader = PdfReader(Data)
    for page in pdf_reader.pages:
      text += page.extract_text()
    texts = text_splitter.split_text(text)
    vectordb.add_texts(texts)
    print("VectorDB updated")
  if Type == 2:
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=300)
    loader = WebBaseLoader(web_paths=(Data,),
                          bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                              class_=("mw-body"))))
    text_documents = loader.load()

    # Convert the list to an iterable
    documents = text_splitter.split_documents(text_documents)
    vectordb.add_documents(documents)
    print("VectorDB updated")
  return vectordb

def save_vectordb(vectordb,Folder_Path):
   vectordb.save_local(folder_path=Folder_Path)
   print("save successful")
def Load_vectordb(Folder_Path,embeddings):
   return FAISS.load_local(folder_path=Folder_Path,embeddings=embeddings,allow_dangerous_deserialization=True)



vectordb = Create_vectordb("/content/Ancient History of Egypt.pdf",embeddings)
print("created vectorDB")
save_vectordb(vectordb,"VectorDB")
vectordb = Load_vectordb("VectorDB",embeddings)
vectordb = Update_VectorDB("VectorDB",embeddings,"https://en.wikipedia.org/wiki/History_of_Egypt",2)
vectordb = Update_VectorDB("VectorDB",embeddings,"https://en.wikipedia.org/wiki/State_of_Palestine",2)
vectordb = Update_VectorDB("VectorDB",embeddings,"https://en.wikipedia.org/wiki/History_of_Palestine",2)
vectordb = Update_VectorDB("VectorDB",embeddings,"https://en.wikipedia.org/wiki/Arabic_history",2)
vectordb = Update_VectorDB("VectorDB",embeddings,"https://en.wikipedia.org/wiki/Arabic_culture",2)
vectordb = Update_VectorDB("VectorDB",embeddings,"https://en.wikipedia.org/wiki/Arabic_religion",2)
save_vectordb(vectordb,"VectorDB")

<ipython-input-20-f7b259c2029c>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings()
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda
2024-12-05 21:06:03.009 INFO    sentence_transformers.SentenceTransformer: Use pytorch device_name: cuda
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transfo

created vectorDB
save successful
VectorDB updated
VectorDB updated
VectorDB updated
VectorDB updated
VectorDB updated
VectorDB updated
save successful


In [ ]:
# print(vectordb.similarity_search("WHAT IS AI"))
print(vectordb.similarity_search("WHAT IS MENTAL HEALTH?"))
print(vectordb.similarity_search("WHAT IS history of egypt"))

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Arabic_religion'}, page_content='Other reasons this message may be displayed:\n\nIf a page was recently created here, it may not be visible yet because of a delay in updating the database; wait a few minutes or try the purge function.\nTitles on Wikipedia are case sensitive except for the first character; please check alternative capitalizations and consider adding a redirect here to the correct title.\nIf the page has been deleted, check the deletion log, and see Why was the page I created deleted?\n\n\n\nRetrieved from "https://en.wikipedia.org/wiki/Arabic_religion"'), Document(metadata={'source': 'https://en.wikipedia.org/wiki/Arabic_religion'}, page_content='Arabic religion\n\n\n\nAdd languages\n\n\n\n\t\t\t\t\t\tPage contents not supported in other languages.\n\t\t\t\t\t\n\n\n\n\n\n\n\n\n\n\nArticleTalk\n\n\n\n\n\nEnglish\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nTools\n\n\n\n\n\nTools\nmove to sidebar\nhide\n\n\n

In [ ]:
import langchain
langchain.__version__

'0.2.11'

In [ ]:
os.environ["GROQ_API_KEY"] = "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"

In [ ]:
qa_chain=load_LLM_groq(vectordb,"llama-3.1-70b-versatile")
prompt = '''
you are an expert historian who knows about egypt ,palastine and arabic history answer the questions as short as possible/n
'''

In [ ]:
# # invoke the qa chain and get a response for user query
query = prompt + """
what is the history of palestine?
"""
response = qa_chain.invoke({"query": query})
print(response["result"])

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-05 21:07:26.645 INFO    httpx: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Palestine's history spans over 5,000 years, with various empires and civilizations ruling the region. Here's a brief overview:

* Ancient Period (3000 BC - 636 AD): Canaanites, Philistines, Israelites, and other tribes inhabited the region.
* Roman and Byzantine Period (636 AD - 638 AD): Palestine was a Roman province, followed by Byzantine rule.
* Islamic Period (638 AD - 1517 AD): Palestine was conquered by Arabs and became a major center of Islamic culture.
* Ottoman Period (1517 AD - 1917 AD): Palestine was under Ottoman rule, with a significant Jewish population.
* British Mandate (1917 AD - 1948 AD): Palestine was under British control, with the 1917 Balfour Declaration supporting a Jewish homeland.
* 1948: Israel declared independence, leading to the displacement of Palestinians (Nakba) and ongoing conflict.

This is a condensed version of a complex history.


In [ ]:
query = prompt + """
اخبرني عن اسرئيل
"""
response = qa_chain.invoke({"query": query})
print(response["result"])

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-05 21:16:23.519 INFO    httpx: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


لا أملك معلومات كافية عن تاريخ إسرائيل، ولكن يمكنني أن أخبرك أن تاريخ إسرائيل الحديث يبدأ بعد عام 1948، عندما أعلنت إسرائيل استقلالها عن الانتداب البريطاني على فلسطين.


In [ ]:
%%writefile app.py -a
import streamlit as st

vectordb= Load_vectordb("VectorDB",embeddings)
prompt = '''
you are an expert historian who knows about egypt ,palastine and arabic history answer the questions as short as possible/n
'''
# Title of the Streamlit app
st.title("LLM RAG Chatbot")

# Selection box to choose between OpenAI and Groq models
model_option = st.selectbox('Choose a model:', ('OpenAI', 'Groq'))

# Input box for user query
user_input = st.text_input("Ask your question:")
user_input = prompt + user_input
# Add OpenAI and Groq API keys to the environment
os.environ["OPENAI_API_KEY"] = "your_openai_api_key"
os.environ["GROQ_API_KEY"] = "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"

# Button to submit the query
if st.button('Get Answer'):

    # Check which model the user selected
    if model_option == 'OpenAI':
        # Load OpenAI model (for example: GPT-4)
        qa_chain = load_LLM_openai(vectordb, model_name="gpt-4o")
        response = qa_chain.invoke({"query": user_input})
        st.write(f"OpenAI's response: {response['result']}")

    elif model_option == 'Groq':
        # Load Groq model
        qa_chain=load_LLM_groq(vectordb,"llama-3.2-11b-text-preview")
        response = qa_chain.invoke({"query": user_input})
        st.write(f"Groq's response: {response['result']}")


Appending to app.py


In [ ]:
%%writefile app.py -a
import streamlit as st
# Initialize session state to store chat history and API keys
if "messages" not in st.session_state:
    st.session_state["messages"] = []
if "openai_api_key" not in st.session_state:
    st.session_state["openai_api_key"] = ""
if "groq_api_key" not in st.session_state:
    st.session_state["groq_api_key"] = ""

# Sidebar for Model Selection and API Key Input
st.sidebar.title("Configuration")

# API Key Input Fields
st.sidebar.subheader("Enter API Keys")
st.session_state["openai_api_key"] = st.sidebar.text_input("OpenAI API Key", type="password")
st.session_state["groq_api_key"] = st.sidebar.text_input("Groq API Key", type="password")

# Model Selection Dropdown
model_option = st.sidebar.selectbox('Choose a model:', ('OpenAI', 'Groq'))

# Set API keys in environment variables if provided

# Load FAISS vectordb (assuming you have already created the FAISS index)
vectordb= Load_vectordb("VectorDB",embeddings)
prompt = '''
you are an expert historian who knows about egypt ,palastine and arabic history answer the questions as short as possible/n
'''

# Chatbot Interface: Title and Chat History
st.title("LLM RAG Chatbot")

# Display chat history
st.write("### Chat History:")
for message in st.session_state["messages"]:
    st.write(message)

# Input box for user query
user_input = st.text_input("You:", key="user_input")

# Button to send the query
if st.button('Send'):
    # Append user input to chat history
    st.session_state["messages"].append(f"You: {user_input}")
    user_input = prompt + user_input

    # Ensure that the appropriate API key is provided
    if model_option == 'OpenAI' and not st.session_state["openai_api_key"]:
        st.error("Please enter your OpenAI API key.")
    elif model_option == 'Groq' and not st.session_state["groq_api_key"]:
        st.error("Please enter your Groq API key.")
    else:
        # Process the query using the selected model
        if model_option == 'OpenAI':
          if st.session_state["openai_api_key"]:
            os.environ["OPENAI_API_KEY"] = st.session_state["openai_api_key"]
            qa_chain = load_LLM_openai(vectordb, model_name="gpt-4")
            response = qa_chain({"query": user_input})
            bot_response = f"OpenAI: {response['answer']}"

        elif model_option == 'Groq':
          if st.session_state["groq_api_key"]:
            os.environ["GROQ_API_KEY"] = st.session_state["groq_api_key"]
            qa_chain=load_LLM_groq(vectordb,"llama-3.2-90b-text-preview")
            response = qa_chain({"query": user_input})
            bot_response = f"Groq: {response['result']}"

        # Append the bot's response to the chat history
        st.session_state["messages"].append(bot_response)

        # Clear input box and refresh chat
        st.experimental_rerun()  # Refresh to display the new message


Appending to app.py


In [ ]:
!npm install -g localtunnel


⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦
changed 22 packages in 2s
⠦
⠦3 packages are looking for funding
⠦  run `npm fund` for details
⠦

In [ ]:
!wget -q -O - ipv4.icanhazip.com

35.185.230.111


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501

⠙⠹⠸⠼⠴⠦⠧


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.185.230.111:8501

your url is: https://ripe-pandas-wait.loca.lt
  Stopping...
^C
